# Imports

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sbn
import sklearn as sk
import tensorflow_probability as tfp
import gresearch_crypto
import warnings
import time
import gc


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit
from keras.layers import Dense, Conv1D, BatchNormalization, LSTM, Bidirectional, LeakyReLU, Activation, MaxPooling1D, Concatenate, Dropout, Conv2D, Masking, Lambda, AveragePooling2D, AveragePooling1D
from keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from keras import backend as K
from keras.utils.vis_utils import plot_model

warnings.filterwarnings("ignore")
tf.autograph.set_verbosity(0)

train = pd.read_csv('../input/g-research-crypto-forecasting/train.csv')
asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
asset_ids = list(train["Asset_ID"].unique())
asset_ids.sort()
models = {}

# Misc

In [ ]:
def Plot_Results(history, y_test, y_pred):
    fig ,axs = plt.subplots(1, 2, figsize=(10, 5))
        
    axs[0].scatter(y_test, y_pred)
    axs[0].legend()
    
    axs[1].plot(history.history["loss"])
    axs[1].plot(history.history["val_loss"])
    axs[1].legend()

    plt.show()
    
def Reduce_Memory_Usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
         
def Compute(df): 
    R=list()
    c=list(df['Close'])
    for i in range(df.shape[0]):
        future=c[min([i+16,df.shape[0]-1])]
        past=c[min([i+1,df.shape[0]-1])]
        R.append(future/past)
        
    R=np.array(R)
    df['pred']=R-1
    return df

def Calculate_Target(df):
    crops=list()
    for a in range(14):
        crops.append(Compute(df[df['Asset_ID']==a]))
        
    conc=pd.concat(crops)
    conc=conc.reset_index()
    j=np.array(list(conc['Target'].isnull()))
    new_targets=np.where(j,conc['pred'],conc['Target'])
    conc['Target']=new_targets
    conc=conc.drop(columns=['pred', 'index'],axis=1)
    return conc

def Common_Timestamps(df):
    timestamps = train['timestamp'].value_counts()==14
    timestamps = timestamps[timestamps]
    df = df.loc[df['timestamp'].isin(timestamps.index)]
    
    return df

train = Reduce_Memory_Usage(train)
train = Common_Timestamps(train)
train = Calculate_Target(train)

# EDA

In [ ]:
def EDA(df):              
    fig ,axs = plt.subplots(len(asset_ids), figsize=(15, 20))
    for asset_id in asset_ids: 
        sbn.lineplot(ax=axs[asset_id], data=train[train['Asset_ID']==asset_id], x="timestamp", y="Close")
#EDA(train)

# Parameters

In [ ]:
n_samples = 100000
n_assets = 14
n_columns = 20
n_timesteps = 1

# Loss Function

In [ ]:
def correlation_coefficient_loss(y_true, y_pred):
    x = y_true
    y = y_pred
    
    mx = tf.reduce_mean(x)
    my = tf.reduce_mean(y)

    xm, ym = x-mx, y-my

    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))

    r = r_num / r_den
 
    r = K.maximum(K.minimum(r, 1.0), -1.0)

    return 1 - K.square(r)

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Model

In [ ]:
def Model(n_assets, n_timesteps, n_columns):
    outs = []
    input = keras.layers.Input(shape=(n_assets, n_timesteps, n_columns))

    for i in range(n_assets):
        inp = Lambda(lambda x: x[:,i,:,:])(input)
        x = Masking(mask_value=0)(inp)   
        x = Conv1D(1024, 1)(x)
        x = LSTM(256, return_sequences=True)(x)
        x = BatchNormalization()(x)
        x = Activation(keras.activations.relu)(x)
        
        x = LSTM(128, return_sequences=True)(x)
        x = BatchNormalization()(x)
        x = Activation(keras.activations.relu)(x)
        outs.append(x)

    out = Concatenate(axis=1)(outs)
    out = Conv1D(2048, 14)(out)
    
    out = Dense(512)(out)
    out = Activation(keras.activations.elu)(out)
    
    out = Dense(256)(out)
    out = Activation(keras.activations.elu)(out)
    
    out = Dense(128)(out)
    out = Activation(keras.activations.elu)(out)
    
    out = Dense(64)(out)
    out = Activation(keras.activations.elu)(out)
    
    out = Dense(32)(out)
    out = Activation(keras.activations.elu)(out)
    
    out = Dense(14)(out)
    out = Activation(keras.activations.linear)(out)
    
    model = tf.keras.Model(inputs=input, outputs=out)  

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss=correlation_coefficient_loss, optimizer=optimizer)
              
    return model

plot_model(Model(n_assets, n_timesteps, n_columns))

# PreProcessing

In [ ]:
def Pre_Processing(df): 
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    
    Add_Features(df)
       
def Add_Features(df):
    #Quarter/One Hot Encode
    x = df['timestamp'].astype('datetime64[s]')
    ohe = keras.utils.to_categorical(x.dt.quarter, num_classes=5, dtype=np.int8)
    df['Q1'] = ohe[:, 1]
    df['Q2'] = ohe[:, 2]
    df['Q3'] = ohe[:, 3]
    df['Q4'] = ohe[:, 4]       
    #Market Cap
    df['market_cap'] = df['Asset_ID'].map(asset_details.set_index('Asset_ID')['Weight'])
    #Close-Open
    df['price_change'] = df['Close'] - df['Open'].astype(np.float16)
    #High-Low
    df['spread'] = df['High'] - df['Low'].astype(np.float16)
    #Upper Shadow
    df['upper_shadow'] = df['High'] - np.maximum(df['Open'], df['Close'])
    #Lower Shadow
    df['lower_shadow'] = np.minimum(df['Open'], df['Close']) - df['Low']
    #Mean Trade
    df['mean_trade'] = df['Volume'] / df['Count']
    #Log Price Change
    df['log_price_change'] = np.log(df['Close']/df['Open'])
    
Pre_Processing(train)

# Training

In [ ]:
def Train_Model(df): 
    scaler = MinMaxScaler()
    X = np.empty((n_samples, n_assets, n_timesteps, n_columns))
    Y = np.empty((n_samples, n_assets, n_timesteps, 1))

    for asset_id in asset_ids:
        asset = df.loc[df['Asset_ID']==asset_id]
        x = asset.loc[:, df.columns != 'Target'].to_numpy()
        x = scaler.fit_transform(x)
        y = asset['Target'].to_numpy()
        for i in range(n_samples):
            temp = x[i,:]
            temp = temp.reshape(1, temp.shape[0])
            X[i, asset_id,:,:] = temp
            
            temp = y[i]
            temp = temp.reshape(1, 1)
            Y[i, asset_id,:,:] = temp
  
    model = Model(n_assets, n_timesteps, n_columns)
              
    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=0, min_delta=0.0001, mode="min", restore_best_weights=True)
    cp = tf.keras.callbacks.ModelCheckpoint(r'./model.h5', save_best_only=True, monitor='val_loss', mode='min')
    rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='min')

    tscv = TimeSeriesSplit(5)
    i=0
    for train_index, test_index in tscv.split(X):
        print("Fold " + str(i) + " ---------------------------------------------------")
        i += 1
        X_train, X_val = X[train_index,:,:,:], X[test_index,:,:,:]
        y_train, y_val = Y[train_index,:,:,:], Y[test_index,:,:,:]
            
        
        history = model.fit(X_train, y_train,
                            validation_data=(X_val, y_val),
                            epochs=500,
                            batch_size=32,
                            callbacks =[es, cp])
        
        
        y_pred = model.predict(X_val)
        Plot_Results(history, y_val, y_pred)
    
    return model

model = Train_Model(train) 

# Submission 

In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

t0 = time.time()
for i, (df_test, df_pred) in enumerate(iter_test):    
    df_test = Reduce_Memory_Usage(df_test)
    Pre_Processing(df_test)
  
    df_test = df_test.drop('row_id', axis=1)
    df_test = df_test.to_numpy().reshape(-1, n_assets, n_timesteps, n_columns)

    df_pred['Target'] = model.predict(df_test).squeeze()
    env.predict(df_pred)
t1 = time.time() - t0
print(t1)